In [1]:
from bs4 import BeautifulSoup
import urllib.request
import csv
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import csv

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Initialization Complete!')

Initialization Complete!


### GET THE POSTAL CODE INFORMATION FROM THE WEBSITE

In [9]:
#scrape the page using beautifulsoup
page = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = (BeautifulSoup(page, 'lxml'))
#get the table
pc_table = soup.find('table',attrs={"class":"wikitable sortable"})
#get the rows
pc_table_rows = pc_table.select('tbody > tr')
#get the header
header = [th.text.rstrip() for th in pc_table_rows[0].find_all('th')]
#write the rows info to the csv file
with open('to_pc.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(header)
    for row in pc_table_rows[1:]:
        data = [th.text.rstrip() for th in row.find_all('td')]
        writer.writerow(data)

### Write out to file (Postal Codes with Borough and Neighborhood)

In [10]:
df = pd.read_csv('to_pc.csv')
df.dropna(axis=0,inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(15)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [11]:
df.to_csv('toronto_pc.csv',index=False)

In [12]:
df = pd.read_csv('toronto_pc.csv')
df1 = pd.read_csv('geospatial_coord.csv')

#merge the two files using the postal code as key
df_merge = pd.merge(left=df, right=df1,how='left',left_on='Postal Code',right_on='Postal Code')

### Write out file (with Postal Code, Borough, Neighborhood and Lat/Long values)

In [13]:
df_merge.to_csv('toronto_pc_ll1.csv',index=False)

In [14]:
df2 = pd.read_csv('toronto_pc_ll1.csv')
df2.head(15)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [15]:
df2.shape

(103, 5)